# Quick Start

Generate binary forecasting questions from news articles in just a few steps.

> **Tip**: See [API.md](../API.md) for complete API reference and alternative configurations.

## Step 1: Install SDK

In [1]:
%pip install -e ../

from IPython.display import clear_output
clear_output()

## Step 2: Import and Initialize Client

The `LightningRod` client provides access to transforms, datasets, files, and filesets. See [API.md](../API.md) for all available methods.

In [ ]:
import os
from lightningrod import LightningRod

api_key = os.getenv("LIGHTNINGROD_API_KEY")
if not api_key:
    raise ValueError("LIGHTNINGROD_API_KEY is not set")

client = LightningRod(api_key=api_key)

## Step 3: Configure Seed Generator

Searches Google News for articles matching your query.

> **Alternatives**: Use `GdeltSeedGenerator` for large-scale datasets, or `FileSetSeedGenerator` for custom documents. See [API.md](../API.md) for details.

In [3]:
from datetime import datetime, timedelta
from lightningrod import NewsSeedGenerator

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    interval_duration_days=7,
    search_query="technology announcements",
)

## Step 4: Configure Question Generator

Creates forecasting questions from the news articles.

> **Note**: We use `BINARY` answer type here. Other types: `CONTINUOUS` (numeric), `MULTIPLE_CHOICE`, `FREE_RESPONSE`. See examples 05-08.

In [4]:
from lightningrod import QuestionGenerator, AnswerType, AnswerTypeEnum

answer_type = AnswerType(answer_type=AnswerTypeEnum.BINARY)

question_generator = QuestionGenerator(
    instructions="Generate forward-looking questions about technology announcements.",
    answer_type=answer_type,
)

## Step 5: Configure Labeler and Renderer

Labeler finds answers using web search. Renderer formats the final prompt.

> **Note**: The renderer populates the `prompt` field in each sample. See [API.md](../API.md) for sample structure.

In [5]:
from lightningrod import WebSearchLabeler
from lightningrod._generated.models import QuestionRenderer

labeler = WebSearchLabeler(answer_type=answer_type)
renderer = QuestionRenderer(answer_type=answer_type)

## Step 6: Create Pipeline and Run

Combine all components and run to generate your dataset.

> **Alternative**: Use `client.transforms.submit()` to submit without waiting, then check status with `client.transforms.jobs.get(job_id)`. See [API.md](../API.md).

In [6]:
from lightningrod import QuestionPipeline

pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=renderer,
)

dataset = client.transforms.run(pipeline_config, max_questions=10)

## Step 7: View Results

Inspect the generated questions and answers. Each sample contains `seed`, `question`, `label`, `prompt`, and optional `context` and `meta` fields. See [API.md](../API.md) for the complete sample structure.

In [7]:

%pip install pandas

from IPython.display import clear_output
clear_output()

In [8]:
import pandas as pd
from typing import List
from lightningrod import Sample

def samples_to_dataframe(samples: List[Sample]) -> pd.DataFrame:
    df_data = []
    for sample in samples:
        row = {}
        if sample.question:
            row['question'] = sample.question.question_text
        if sample.label:
            row['answer'] = sample.label.label
            row['confidence'] = sample.label.label_confidence
        if sample.prompt:
            row['prompt'] = sample.prompt
        if sample.seed:
            row['seed_text'] = getattr(sample.seed, 'seed_text', None)
        df_data.append(row)
    return pd.DataFrame(df_data)

samples = dataset.download()

print(f"Generated {dataset.num_rows} samples\n")

df = samples_to_dataframe(samples)
print(df.head())

Generated 10 samples

                                            question        answer  \
0  Will Andhra Pradesh make major investment anno...             1   
1  Will a major American professional sports leag...             1   
2  Will Accesso Technology Group PLC announce an ...  Undetermined   
3  Will Breton Technology’s controlling sharehold...             0   
4  Will UMT United Mobility Technology successful...  Undetermined   

   confidence                                             prompt  \
0         1.0  QUESTION:\nWill Andhra Pradesh make major inve...   
1         1.0  QUESTION:\nWill a major American professional ...   
2         0.9                                                NaN   
3         0.9  QUESTION:\nWill Breton Technology’s controllin...   
4         1.0                                                NaN   

                                           seed_text  
0  Title: Andhra Pradesh Courts Global Investors ...  
1  Title: AI-Generated Fake Sports New